In [1]:
!pip install dash==1.19.0
!pip install jupyter_dash 
!pip install --upgrade plotly

from jupyter_dash import JupyterDash

Requirement already up-to-date: plotly in c:\users\user\anaconda3\lib\site-packages (5.13.1)


In [2]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go 

Running in Jupyter, call the following function to detect the proxy configuration.

In [3]:
file_loc = "https://raw.githubusercontent.com/Barbj379/Two_Ton/main/Item%20Sales%20Report.csv"

df = pd.read_csv(file_loc)

In [4]:
df.head()

,Item Name,Global Category,Major Category,Minor Category,Item Quantity,Item/Cover %,Average Item Price,Gross Sales,Net Sales
0,robot insurance 10oz,BEVERAGE,ALCOHOL,10OZ DRAUGHT,96.0,6.55,7.0,672.0,665.0
1,knotty pretzels: italian,FOOD,FOOD,FOOD,9.0,0.61,7.0,63.0,63.0
2,do what's in your heart (10oz),BEVERAGE,ALCOHOL,10OZ DRAUGHT,85.0,5.80,7.0,595.0,595.0
3,field artillery 10oz,BEVERAGE,ALCOHOL,10OZ DRAUGHT,84.0,5.73,8.0,672.0,672.0
4,socially acceptable,BEVERAGE,ALCOHOL,10OZ DRAUGHT,8.0,0.55,6.0,48.0,48.0


#### Explore dataset

In [5]:
df.groupby(['Minor Category'])['Net Sales'].mean()

Minor Category
10OZ DRAUGHT           399.806190
16OZ DRAUGHT          1042.561333
32OZ GROWLER FILLS      23.250000
4OZ DRAUGHT             26.823529
64OZ GROWLER FILLS     153.815789
BEER FLIGHTS          1771.626667
CLOTHING                45.400000
FOOD                    47.230769
GIFT CARDS             871.000000
KEGS                   110.000000
MERCHANDISE             74.500000
PACKAGED GOODS         558.500000
PRIVATE EVENTS        1750.000000
Name: Net Sales, dtype: float64

In [6]:
df.groupby(['Major Category'])['Net Sales'].sum()

Major Category
ALCOHOL            33023.73
FOOD                 614.00
GIFT CARD SALES      871.00
RETAIL              5477.00
Name: Net Sales, dtype: float64

#### Create a dash application

In [7]:
# Run this app with `python app.py` and
#visit http://127.0.0.1:8050/ in your web browser.
from jupyter_dash import JupyterDash
import dash
from dash import Dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

file_loc0 = "https://raw.githubusercontent.com/Barbj379/Two_Ton/main/Item%20Sales%20Report.csv"
file_loc1 = "https://raw.githubusercontent.com/Barbj379/Two_Ton/main/Profit_Loss.csv"

df_menu = pd.read_csv(file_loc0)
df_pl = pd.read_csv(file_loc1)

category_list = [i for i in df_menu['Major Category'].unique()]
items = df_menu['Major Category'].unique()

app = JupyterDash(__name__)
server = app.server

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

"""Compute graph data for creating beer sales report
Function that takes sales data as input and creates 3 dataframes based on the grouping condition
to be used for plotting the charts and graphs.

Argument:
    df: Filtered dataframe
Returns:
    Dataframes to create graph.

"""

def compute_data_choice(df_menu):
    #Total sales by Item Name
    tree_data = df_menu.groupby(['Minor Category', 'Item Name'])['Net Sales'].sum().reset_index()
    #Profit by month
    bar_data = df_pl.groupby(['Date'])['Amount Saved'].sum().reset_index()
    #Percentage of overall sales
    scatter_data = df_menu.groupby(['Minor Category', 'Item Name'])['Item/Cover %'].sum().reset_index()
    return tree_data, bar_data, scatter_data
    
app.layout = html.Div(children=[
        html.H1(children="Menu Item Sales, Past Three Months",
                   style={'textAlign': 'center', 'color':'#503D36', 'font-size': 36}),
        html.Div([
            html.Div([
                html.Div(
                    [html.H2('Major Category Type', style={'margin-right': '2em'}),
                         ]),
                      dcc.Dropdown(id='input-type', 
                      options=[{'label': i, 'value': i} for i in category_list],
                                   searchable = False,
                                  value=items,
                           style={'width': '80%',
                                  'height':'3px', 
                                  'font-size': 18, 
                                  'textAlign': 'center'}),
            ], style={'display':'flex'}),
    
        
            
        html.Div([ ], id='plot1'),            
        html.Div([
            html.Div([ ], id='plot2'),
            html.Div([ ], id='plot3')],
            style ={'display': 'flex'})
        ])
])
        



  #Define callback to update graph
@app.callback([
    Output(component_id='plot1', component_property='children'),
    Output(component_id='plot2', component_property='children'),
    Output(component_id='plot3', component_property='children')],
    [Input(component_id='input-type', component_property='value') ],
     [State("plot1", 'children'), State("plot2", "children"),
             State("plot3", "children")]
)
    
    
# Add computation to callback function and return graph
def get_graph(chart, children1, children2, children3):   
    
    if chart == 'values':
    
        # Compute required information for creating graph from the data
        tree_data, bar_data, scatter_data = compute_data_choice(df_menu, df_pl)
    
        #Percentage of overall sales
        scatter_fig = px.scatter(scatter_data, x="Item Name", y="Item/Cover %",
                 size="Net Sales")
        bar_fig = px.bar(bar_data, x='Item Name'[0:25], y='Net Sales', title='Net Sales by Product')
    
        tree_fig = px.treemap(tree_data, path=['Minor Category', 'Net Sales', 'Item Name'],
                         values='Net Sales', color='Net Sales', color_continuous_scale='RdBu',
                         title= 'Net Sales by Craft Beer Product in USD',
                          hover_data=['Item Name']
                                    )
    
        return [dcc.Graph(figure=tree_fig),
                dcc.Graph(figure=bar_fig),
                dcc.Graph(figure=scatter_fig)],

#app.Select-menu-outer{display : block !important;},
    
    
#Run app and display result inline in the notebook
app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
